In [1]:
import networkx as nx
import pickle
import os

In [2]:
directory = os.getcwd()
filename1 = directory + "/grafi/graph_V1500_E10000.pickle"

In [4]:
G = pickle.load(open(filename1, 'rb'))
matrix = nx.to_numpy_matrix(G)
print(matrix)

AttributeError: module 'networkx' has no attribute 'to_numpy_matrix'